# Sistemas de Recomendação baseado em contexto

Neste trabalho será apresentado um exemplo de sistema de recomendação baseado em contexto, mais especificamente usando o contexto de localização.

As técnicas de sistemas de recomendação baseado em contexto são divididas em 3 categorias: Pré-filtro contextual, Pós-filtro contextual e modelagem contextual. Usaremos pra este sistema o Pré-filtro.

Vamos começar carregando o nosso dataset. O dataset utilizado é um disponibilizado pela prefeitura de fortaleza.

In [1]:
# Import Pandas
import pandas as pd

# Load Movies Metadata
metadata = pd.read_csv('./address_dataset/PRACAS_E_PARQUES_PMF.csv',encoding = "ISO-8859-1", low_memory=False)

# Print the first three rows
metadata.head(10)

,x,y,gid,Name,description
0,"-38,5571388404","-3,7234340059",1,AÇUDE JOÃO LOPES - SER I,NaN
1,"-38,4734803598","-3,7504060328",2,CAMINHO DAS CAJARANAS - SER II,NaN
2,"-38,4734112098","-3,7501902415",3,CAMINHO DAS CAJARANAS - SER II,NaN
3,"-38,4735561747","-3,7506311664",4,CAMINHO DAS CAJARANAS - SER II,NaN
4,"-38,4736428836","-3,749141127",5,CAMINHO DAS CAJAZEIRAS - SER II,NaN
5,"-38,4737024653","-3,7493694027",6,CAMINHO DAS CAJAZEIRAS - SER II,NaN
6,"-38,4737825807","-3,7495818118",7,CAMINHO DAS CAJAZEIRAS - SER II,NaN
7,"-38,4742469619","-3,750888257",8,CAMINHO DAS CARAMBOLEIRAS - SER II,NaN
8,"-38,4743875521","-3,7513302915",9,CAMINHO DAS CARAMBOLEIRAS - SER II,NaN
9,"-38,4743158635","-3,7511041738",10,CAMINHO DAS CARAMBOLEIRAS - SER II,NaN


No dataset podemos observar que tem nomes que se repetem, por isso vamos retirar essas duplicatas:

In [2]:
q_parks = metadata.copy()
q_parks = q_parks.drop_duplicates(['Name'], keep='last')
q_parks.head(10)

,x,y,gid,Name,description
0,"-38,5571388404","-3,7234340059",1,AÇUDE JOÃO LOPES - SER I,NaN
3,"-38,4735561747","-3,7506311664",4,CAMINHO DAS CAJARANAS - SER II,NaN
6,"-38,4737825807","-3,7495818118",7,CAMINHO DAS CAJAZEIRAS - SER II,NaN
9,"-38,4743158635","-3,7511041738",10,CAMINHO DAS CARAMBOLEIRAS - SER II,NaN
12,"-38,4697663136","-3,7499202009",13,CAMINHO DAS TANGERINAS - SER II,NaN
15,"-38,472954444","-3,7488792698",16,CAMINHO DOS ABACATEIROS - SER II,NaN
18,"-38,4707380424","-3,7529895791",19,CAMINHO DOS JAMBEIROS - SER II,NaN
21,"-38,4699835502","-3,7522678653",22,CAMINHO DOS LIMOEIROS - SER II,NaN
23,"-38,4695560943","-3,7509494568",24,CAMINHO DOS SAPOTIZEIROS - SER II,NaN
24,"-38,5026734815","-3,7306021469",25,CAMPO DO AMÉRICA - PRAÇA SOARES JUAREZ - SER II,NaN


A idéia da recomendação é encontrar o parque/praça mais segura porém ainda não temos informações do grau de segurança do parque. por causa disso vamos colocar notas de segurança aleatórias que variam de 1-5, sendo 1-Perigoso e 5-Seguro.

In [3]:
import numpy as np
q_parks['Security'] = np.random.randint(1,6, size=len(q_parks))
q_parks.head(10)

,x,y,gid,Name,description,Security
0,"-38,5571388404","-3,7234340059",1,AÇUDE JOÃO LOPES - SER I,NaN,5
3,"-38,4735561747","-3,7506311664",4,CAMINHO DAS CAJARANAS - SER II,NaN,3
6,"-38,4737825807","-3,7495818118",7,CAMINHO DAS CAJAZEIRAS - SER II,NaN,4
9,"-38,4743158635","-3,7511041738",10,CAMINHO DAS CARAMBOLEIRAS - SER II,NaN,4
12,"-38,4697663136","-3,7499202009",13,CAMINHO DAS TANGERINAS - SER II,NaN,4
15,"-38,472954444","-3,7488792698",16,CAMINHO DOS ABACATEIROS - SER II,NaN,2
18,"-38,4707380424","-3,7529895791",19,CAMINHO DOS JAMBEIROS - SER II,NaN,3
21,"-38,4699835502","-3,7522678653",22,CAMINHO DOS LIMOEIROS - SER II,NaN,2
23,"-38,4695560943","-3,7509494568",24,CAMINHO DOS SAPOTIZEIROS - SER II,NaN,3
24,"-38,5026734815","-3,7306021469",25,CAMPO DO AMÉRICA - PRAÇA SOARES JUAREZ - SER II,NaN,5


In [4]:
q_parks.shape

(167, 6)

Agora podemos ver que no nosso dataset temos no total de 167 parques/praças em Fortaleza.

O próximo passo será colocar no dataset as informações de latitude e longitude dos parques/praças.

In [5]:
q_parks["Lat"] = q_parks["y"]
q_parks["Long"] = q_parks["x"]
q_parks["Lat"] = q_parks["Lat"].replace(to_replace=',', value='.', regex=True)
q_parks["Long"] = q_parks["Long"].replace(to_replace=',', value='.', regex=True)
q_parks.head(10)

,x,y,gid,Name,description,Security,Lat,Long
0,"-38,5571388404","-3,7234340059",1,AÇUDE JOÃO LOPES - SER I,NaN,5,-3.7234340059,-38.5571388404
3,"-38,4735561747","-3,7506311664",4,CAMINHO DAS CAJARANAS - SER II,NaN,3,-3.7506311664,-38.4735561747
6,"-38,4737825807","-3,7495818118",7,CAMINHO DAS CAJAZEIRAS - SER II,NaN,4,-3.7495818118,-38.4737825807
9,"-38,4743158635","-3,7511041738",10,CAMINHO DAS CARAMBOLEIRAS - SER II,NaN,4,-3.7511041738,-38.4743158635
12,"-38,4697663136","-3,7499202009",13,CAMINHO DAS TANGERINAS - SER II,NaN,4,-3.7499202009,-38.4697663136
15,"-38,472954444","-3,7488792698",16,CAMINHO DOS ABACATEIROS - SER II,NaN,2,-3.7488792698,-38.472954444
18,"-38,4707380424","-3,7529895791",19,CAMINHO DOS JAMBEIROS - SER II,NaN,3,-3.7529895791,-38.4707380424
21,"-38,4699835502","-3,7522678653",22,CAMINHO DOS LIMOEIROS - SER II,NaN,2,-3.7522678653,-38.4699835502
23,"-38,4695560943","-3,7509494568",24,CAMINHO DOS SAPOTIZEIROS - SER II,NaN,3,-3.7509494568,-38.4695560943
24,"-38,5026734815","-3,7306021469",25,CAMPO DO AMÉRICA - PRAÇA SOARES JUAREZ - SER II,NaN,5,-3.7306021469,-38.5026734815


Organizando os índices:

In [6]:
q_parks = q_parks.reset_index(drop=True)
q_parks.head(10)

,x,y,gid,Name,description,Security,Lat,Long
0,"-38,5571388404","-3,7234340059",1,AÇUDE JOÃO LOPES - SER I,NaN,5,-3.7234340059,-38.5571388404
1,"-38,4735561747","-3,7506311664",4,CAMINHO DAS CAJARANAS - SER II,NaN,3,-3.7506311664,-38.4735561747
2,"-38,4737825807","-3,7495818118",7,CAMINHO DAS CAJAZEIRAS - SER II,NaN,4,-3.7495818118,-38.4737825807
3,"-38,4743158635","-3,7511041738",10,CAMINHO DAS CARAMBOLEIRAS - SER II,NaN,4,-3.7511041738,-38.4743158635
4,"-38,4697663136","-3,7499202009",13,CAMINHO DAS TANGERINAS - SER II,NaN,4,-3.7499202009,-38.4697663136
5,"-38,472954444","-3,7488792698",16,CAMINHO DOS ABACATEIROS - SER II,NaN,2,-3.7488792698,-38.472954444
6,"-38,4707380424","-3,7529895791",19,CAMINHO DOS JAMBEIROS - SER II,NaN,3,-3.7529895791,-38.4707380424
7,"-38,4699835502","-3,7522678653",22,CAMINHO DOS LIMOEIROS - SER II,NaN,2,-3.7522678653,-38.4699835502
8,"-38,4695560943","-3,7509494568",24,CAMINHO DOS SAPOTIZEIROS - SER II,NaN,3,-3.7509494568,-38.4695560943
9,"-38,5026734815","-3,7306021469",25,CAMPO DO AMÉRICA - PRAÇA SOARES JUAREZ - SER II,NaN,5,-3.7306021469,-38.5026734815


Agora temos o nosso dataset pronto para a recomendação sensível ao contexto.

Nesse exercício vamos utilizar apenas o contexto de localização. A idéia seria o sistema pegar a localização do usuário e a partir dela encontrar todos o parques/praças dentro de 2 km.

Para facilitar nossos testes vamos tentar usar uma localização aleatória que tem as coordenadas dentro desse limite: 
Latitude: -3,68 até -3,86
Longitude: -38,43 até -38,62

In [7]:
import random

coords = (round(random.uniform(-3.86,-3.68), 10),round(random.uniform(-38.62,-38.43), 10))
print(coords)

(-3.8099396272, -38.5510458083)


Sabendo a nossa localização, agora é possível fazer a filtragem do nosso dataset para apenas analisar os parques/praças perto do usuário.

In [8]:
from geopy.distance import vincenty
def distance(a,b):
    return vincenty(a,b).km

In [9]:
dist=[]
for index, row in q_parks.iterrows():
    dist.append(distance(coords,(row.Lat,row.Long)))

q_parks['Dist'] = dist

In [10]:
q_parks = q_parks.drop(q_parks[q_parks.Dist > 2].index)
q_parks.head(10)

,x,y,gid,Name,description,Security,Lat,Long,Dist
52,"-38,5533274324","-3,8079441853",225,PRAÇA CLÓVIS BEVILÁQUA - SER VI,NaN,2,-3.8079441853,-38.5533274324,0.336030
55,"-38,5520274963","-3,8243211549",228,PRAÇA DA CISTERNA - SER V,NaN,2,-3.8243211549,-38.5520274963,1.594032
146,"-38,5498661058","-3,7970214518",324,PRAÇA PRIMEIRO DE MAIO - SER VI,NaN,4,-3.7970214518,-38.5498661058,1.434479
149,"-38,5565475574","-3,8145557417",327,PRAÇA SÃO JOSÉ - SER V,NaN,3,-3.8145557417,-38.5565475574,0.796245


Com isso terminamos a recomendação sensível ao contexto, agora falta utilizar algum outro recomendador para encontrar a lista de parques seguros

### Recomendador Simples

Agora vamos ordenar pela segurança

In [11]:
q_parks = q_parks.sort_values('Security', ascending=False)

q_parks

,x,y,gid,Name,description,Security,Lat,Long,Dist
146,"-38,5498661058","-3,7970214518",324,PRAÇA PRIMEIRO DE MAIO - SER VI,NaN,4,-3.7970214518,-38.5498661058,1.434479
149,"-38,5565475574","-3,8145557417",327,PRAÇA SÃO JOSÉ - SER V,NaN,3,-3.8145557417,-38.5565475574,0.796245
52,"-38,5533274324","-3,8079441853",225,PRAÇA CLÓVIS BEVILÁQUA - SER VI,NaN,2,-3.8079441853,-38.5533274324,0.336030
55,"-38,5520274963","-3,8243211549",228,PRAÇA DA CISTERNA - SER V,NaN,2,-3.8243211549,-38.5520274963,1.594032
